# Demo of Multi-site Agentic Framework #

Same as before, imports, but will be using CrewAI framework

In [3]:
import os
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task

Create the language model's to be used

In [14]:
# Initialize LLM objects for our custom endpoints
llm_eu = LLM(
    model="openai/Qwen2.5-7B-Instruct-GGUF",
    base_url="http://172.18.0.4:61100/v1",
    api_key="local"
)

llm_usa = LLM(
    model="openai/Qwen2.5-7B-Instruct-GGUF",
    base_url="http://localhost:61102/v1",
    api_key="local"
)

Define the crew

In [15]:
# Agents
eu_agent = Agent(
    role="Creative European Story Topic Generator",
    goal="Generate ONE creative and interesting story topic",
    backstory=(
        "You are a creative European storyteller with a flair for imaginative and unique ideas. "
        "Your specialty is generating compelling story topics that spark creativity and engagement. "
        "You focus on imaginative concepts that can form the basis of captivating narratives."
    ),
    llm=llm_eu,
    verbose=True,
    allow_delegation=False
)

usa_agent = Agent(
    role="Creative American Story Writer",
    goal="Write engaging and well-written short stories",
    backstory=(
        "You are a skilled American storyteller who excels at crafting compelling narratives. "
        "You take creative topics and transform them into engaging short stories that captivate readers. "
        "Your stories are concise yet captivating, with strong character development and imaginative plots."
    ),
    llm=llm_usa,
    verbose=True,
    allow_delegation=False
)

# Tasks
generate_topic_task = Task(
    description=(
        "Generate ONE creative and interesting story topic.\n"
        "Respond with JUST the topic, nothing else. No greetings, no explanations, just the topic.\n"
        "Focus on imaginative, engaging, and unique story ideas that would make great short stories."
    ),
    expected_output="A single creative story topic as plain text",
    agent=eu_agent
)

write_story_task = Task(
    description=(
        "Write a creative short story based on the topic generated by the EU agent.\n"
        "Make the story engaging and well-written. Keep it concise but captivating.\n"
        "Focus on character development, imaginative plot, and an engaging narrative."
    ),
    expected_output="A complete short story with beginning, middle, and end",
    agent=usa_agent,
    context=[generate_topic_task]  # pulls the EU agent's output as context
)

In [20]:
# Crew
crew = Crew(
    agents=[eu_agent, usa_agent],
    tasks=[generate_topic_task, write_story_task],
    process=Process.sequential,
    verbose=True,
    memory=False
)

result = crew.kickoff()

# Print the EU agent’s topic
print("\n=== EU Topic ===")
print(generate_topic_task.output)


# Print the final story (last task result)
print("\n=== Final Story ===")
print(result)



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 715cace2-c3ed-4690-a322-ebbeff02f35a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Creative European Story Topic Generator                                                                 │
│                                                                                                                 │
│  Task: Generate ONE creative and interesting story topic.                                                       │
│  Respond with JUST the topic, nothing else. No greetings, no explanations, just the topic.                      │
│  Focus on imaginative, engaging, and unique story ideas that would make great short stories.                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Creative European Story Topic Generator                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  A mysterious old map leads a group of friends to a hidden library beneath the waves in the Mediterranean Sea.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 13501067-3e31-4078-b14c-5981775ffbd6                                                                     │
│  Agent: Creative European Story Topic Generator                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Creative American Story Writer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  It was a crisp autumn evening when four friends, Emma, Leo, Mia, and Theo, decided to embark on a peculiar     │
│  adventure. They stumbled upon an old, tattered map tucked away in a dusty antique shop. The map was inscribed  │
│  with cryptic symbols and a single, intriguing phrase: "To the Library Beyond the Waves."                       │
│                                                                                                                 │
│  Curiosity sparked, the friends decided to follow the map’s mysterious directions. With a mix of excitement     │
│  and trepidation, they set off into the Mediterranean, their path leading them to a remote cove where the map   │
│  seemed to be pointing. As the moon cast its silvery light upon the waves, they noticed something shimmering    │
│  beneath the water’s surface—a flickering light that seemed to beckon them closer.                              │
│                                                                                                                 │
│  Emma, the most adventurous, dove into the cool night water. To their astonishment, a solid metal staircase     │
│  emerged from the depths, leading down into the dark waters below. Without hesitation, they followed the        │
│  staircase into the depths of the sea, their flashlights cutting through the inky darkness.                     │
│                                                                                                                 │
│  They soon found themselves in a vast, cavernous chamber beneath the waves. The walls were lined with           │
│  thousands of books and scrolls, each one encased in glass. The air was thick with the scent of parchment and   │
│  ink, and the sound of water lapping against the stone walls filled the chamber.                                │
│                                                                                                                 │
│  Leo, the tech-savvy one, realized that the library was powered by an ancient, yet incredibly sophisticated,    │
│  system. He discovered a series of buttons and controls along the wall, each marked with a symbol from the map  │
│  they had found. With a mixture of caution and daring, they began to press the buttons, triggering a series of  │
│  lights and sounds throughout the library.                                                                      │
│                                                                                                                 │
│  Mia, the quiet observer, noticed that each book seemed to glow faintly when touched. She carefully examined    │
│  the books, finding that they contained information on long-lost sciences and forgotten arts. She shared her    │
│  discoveries with the others, who were equally fascinated and amazed.                                           │
│                                                                                                                 │
│  Theo, the most level-headed, kept a close watch for any potential dangers. He noticed that the library was     │
│  not just a collection of books, but a repository of knowledge that could change the world. He explained the    │
│  historical significance of the library to the others, making them realize that their discovery was far more    │
│  than a simple treasure.                               

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 95a89fbd-71b0-4c29-bc7a-e10b1dd72511                                                                     │
│  Agent: Creative American Story Writer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 715cace2-c3ed-4690-a322-ebbeff02f35a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: It was a crisp autumn evening when four friends, Emma, Leo, Mia, and Theo, decided to embark on  │
│  a peculiar adventure. They stumbled upon an old, tattered map tucked away in a dusty antique shop. The map     │
│  was inscribed with cryptic symbols and a single, intriguing phrase: "To the Library Beyond the Waves."         │
│                                                                                                                 │
│  Curiosity sparked, the friends decided to follow the map’s mysterious directions. With a mix of excitement     │
│  and trepidation, they set off into the Mediterranean, their path leading them to a remote cove where the map   │
│  seemed to be pointing. As the moon cast its silvery light upon the waves, they noticed something shimmering    │
│  beneath the water’s surface—a flickering light that seemed to beckon them closer.                              │
│                                                                                                                 │
│  Emma, the most adventurous, dove into the cool night water. To their astonishment, a solid metal staircase     │
│  emerged from the depths, leading down into the dark waters below. Without hesitation, they followed the        │
│  staircase into the depths of the sea, their flashlights cutting through the inky darkness.                     │
│                                                                                                                 │
│  They soon found themselves in a vast, cavernous chamber beneath the waves. The walls were lined with           │
│  thousands of books and scrolls, each one encased in glass. The air was thick with the scent of parchment and   │
│  ink, and the sound of water lapping against the stone walls filled the chamber.                                │
│                                                                                                                 │
│  Leo, the tech-savvy one, realized that the library was powered by an ancient, yet incredibly sophisticated,    │
│  system. He discovered a series of buttons and controls along the wall, each marked with a symbol from the map  │
│  they had found. With a mixture of caution and daring, they began to press the buttons, triggering a series of  │
│  lights and sounds throughout the library.                                                                      │
│                                                                                                                 │
│  Mia, the quiet observer, noticed that each book seemed to glow faintly when touched. She carefully examined    │
│  the books, finding that they contained information on long-lost sciences and forgotten arts. She shared her    │
│  discoveries with the others, who were equally fascinated and amazed.                                           │
│                                                                                                                 │
│  Theo, the most level-headed, kept a close watch for any potential dangers. He noticed that the library was     │
│  not just a collection of books, but a repository of knowledge that could change the world. He explained the    │
│  historical significance of the library to the others,


=== EU Topic ===
A mysterious old map leads a group of friends to a hidden library beneath the waves in the Mediterranean Sea.

=== Final Story ===
It was a crisp autumn evening when four friends, Emma, Leo, Mia, and Theo, decided to embark on a peculiar adventure. They stumbled upon an old, tattered map tucked away in a dusty antique shop. The map was inscribed with cryptic symbols and a single, intriguing phrase: "To the Library Beyond the Waves."

Curiosity sparked, the friends decided to follow the map’s mysterious directions. With a mix of excitement and trepidation, they set off into the Mediterranean, their path leading them to a remote cove where the map seemed to be pointing. As the moon cast its silvery light upon the waves, they noticed something shimmering beneath the water’s surface—a flickering light that seemed to beckon them closer.

Emma, the most adventurous, dove into the cool night water. To their astonishment, a solid metal staircase emerged from the depths, leadi